* Collect diverse data. Train on the merged set.
* Tweak loss function.

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

import pytorch_lightning as pl
from pytorch_lightning import Trainer, seed_everything
from pytorch_lightning.loggers.csv_logs import CSVLogger

import optuna

In [2]:
class LSTMClassifier(pl.LightningModule):
    # NOTE: All the bidirectional code can be removed. It is unnecessary for your case.
    def __init__(
        self,
        input_dim, # n_features = 2 here
        hidden_dim,
        output_dim,
        num_layers,
        dropout,
        bidirectional=False,
        learning_rate=1e-3,
        criterion=nn.BCEWithLogitsLoss()
    ):
        super().__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.num_layers = num_layers
        self.bidirectional = bidirectional

        self.dropout = dropout
        self.learning_rate = learning_rate
        self.criterion = criterion

        self.lstm = nn.LSTM(
            input_size=input_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout,
            bidirectional=bidirectional
        )
        if self.bidirectional:
            self.fc = nn.Linear(hidden_dim * 2, output_dim)
        else:
            self.fc = nn.Linear(hidden_dim, output_dim)
        
        self.save_hyperparameters(ignore=['criterion'])

    def forward(self, x):
        # Zeroing out the hidden state and cell state for each batch of sequences (or each of the samples)
        if self.bidirectional:
            h0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_dim).to(self.device)
            c0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_dim).to(self.device)
        else:
            h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).to(self.device)
            c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).to(self.device)

        out, (hidden, _) = self.lstm(x, (h0, c0))
        if self.bidirectional:
            out = self.fc(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1))
        else:
            out = self.fc(hidden[-1,:,:])

        return out

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.learning_rate)

    def training_step(self, batch, batch_idx):
        x, y = batch
        out = self(x)
        out = out.squeeze()
        loss = self.criterion(out, y)

        self.log('train_loss', loss)

        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        out = self(x)
        out = out.squeeze()
        loss = self.criterion(out, y)

        self.log('val_loss', loss)

        return loss

    def test_step(self, batch, batch_idx):
        x, y = batch
        out = self(x)
        out = out.squeeze()
        loss = self.criterion(out, y)
        acc = ((torch.sigmoid(out) > 0.5).float() == y).float().mean()

        self.log('test_loss', loss)
        self.log('test_acc', acc)

        return {'test_loss': loss, 'test_acc': acc}

In [3]:
# Set seed for reproducibility
seed_everything(42)

# Load data from .npz file
dat = np.load("./unshuffled_mix_data.npz")
# NOTE: "unshuffled" in `unshuffled_mix_data` refers to the fact that the data is not shuffled across datasets. Within each dataset, the data is shuffled.
X, y = dat['X'], dat['y'] # Shuffled data

# test_dat = np.load("./standardscaled_test_data.npz")
# X_test, y_test = test_dat['X'], test_dat['y'] # Shuffled test data

# Use train_test_split to split the overall data into train + val sets # NOTE: test_size=0.1 -> 0.15 | Sometimes, a shape mismatch may happen if val.size is not large enough. Modify test_size accordingly. 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, shuffle=True) # random_state=42) -> Not needed since seed_everything is used # 0.2 used earlier
# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, shuffle=True) # random_state=42) -> Not needed since seed_everything is used # 0.2 used earlier
# Use train_test_split to split the training data into train + val sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, shuffle=True) # random_state=42) # 0.2 used earlier

X_train.shape, X_val.shape, X_test.shape, y_train.shape, y_val.shape, y_test.shape
# (batch_size, seq_len, num_features) if batch_first=True

Global seed set to 42


((275, 20, 2), (31, 20, 2), (55, 20, 2), (275,), (31,), (55,))

In [4]:
# Optuna objective function to optimize:
# hidden_dim, -> Minimize
# num_layers, -> Minimize
# dropout, -> Minimize
# learning_rate, -> Maximize
# batch_size, -> Maximize
# max_epochs -> Minimize
def objective(trial):
    # Define hyperparameters # NOTE: Edit these as needed
    hidden_dim = trial.suggest_int('hidden_dim', 16, 128)
    num_layers = trial.suggest_int('num_layers', 1, 3)
    dropout = trial.suggest_uniform('dropout', 0.0, 0.5) # FIXME: Replace with suggest_float
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1) # FIXME: Replace with suggest_float
    batch_size = trial.suggest_categorical('batch_size', [8, 16, 32])
    max_epochs = trial.suggest_int('max_epochs', 10, 30)

    # Define the model
    model = LSTMClassifier(
        input_dim=2,
        hidden_dim=hidden_dim,
        output_dim=1,
        num_layers=num_layers,
        dropout=dropout,
        learning_rate=learning_rate
    )

    # Define the trainer
    trainer = Trainer(
        max_epochs=max_epochs,
        logger=CSVLogger(
            save_dir="./optuna_trials",
            name="optuna",
            version=f'hidden_dim={hidden_dim}_num_layers={num_layers}_dropout={dropout}_learning_rate={learning_rate}_batch_size={batch_size}_max_epochs={max_epochs}'
        ),
        log_every_n_steps=1,
        enable_progress_bar=True,
        # fast_dev_run=True # For debugging
    )

    # Define the datasets & dataloaders
    train_data = TensorDataset(torch.from_numpy(X_train).float(), torch.from_numpy(y_train))
    val_data = TensorDataset(torch.from_numpy(X_val).float(), torch.from_numpy(y_val))
    test_data = TensorDataset(torch.from_numpy(X_test).float(), torch.from_numpy(y_test))
    train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size, pin_memory=True, num_workers=4)
    val_loader = DataLoader(val_data, shuffle=False, batch_size=batch_size, pin_memory=True, num_workers=4)
    test_loader = DataLoader(test_data, shuffle=False, batch_size=batch_size, pin_memory=True, num_workers=4)

    # Train the model
    trainer.fit(model, train_loader, val_loader)

    # Test the model
    trainer.test(dataloaders=test_loader)

    # Return the test accuracy
    return trainer.logged_metrics['test_acc']

In [5]:
# Optimize the hyperparameters
study = optuna.create_study(
    direction='maximize', # NOTE: This is for the objective, i.e. test_acc
    pruner=optuna.pruners.MedianPruner(),
    study_name='lstm_classifier',
)
study.optimize(objective, n_trials=20)

# Print the best hyperparameters
print(f"Best hyperparameters: {study.best_params}")

[I 2023-04-18 02:32:50,583] A new study created in memory with name: lstm_classifier
C:\Users\JeS73\AppData\Local\Temp\ipykernel_28748\3640371521.py:12: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  dropout = trial.suggest_uniform('dropout', 0.0, 0.5)
C:\Users\JeS73\AppData\Local\Temp\ipykernel_28748\3640371521.py:13: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type    

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

`Trainer.fit` stopped: `max_epochs=16` reached.
Restoring states from the checkpoint path at ./optuna_trials\optuna\hidden_dim=44_num_layers=1_dropout=0.36453429202609255_learning_rate=0.002304101951078044_batch_size=32_max_epochs=16\checkpoints\epoch=15-step=144.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from the checkpoint at ./optuna_trials\optuna\hidden_dim=44_num_layers=1_dropout=0.36453429202609255_learning_rate=0.002304101951078044_batch_size=32_max_epochs=16\checkpoints\epoch=15-step=144.ckpt


Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc                    1.0
        test_loss          0.024686737070706757
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[I 2023-04-18 03:29:39,061] Trial 19 finished with value: 1.0 and parameters: {'hidden_dim': 44, 'num_layers': 1, 'dropout': 0.36453429202609255, 'learning_rate': 0.002304101951078044, 'batch_size': 32, 'max_epochs': 16}. Best is trial 3 with value: 1.0.


Best hyperparameters: {'hidden_dim': 42, 'num_layers': 1, 'dropout': 0.010454300823487694, 'learning_rate': 0.05738663557134362, 'batch_size': 8, 'max_epochs': 18}


In [10]:
print(f"Best trial: {study.best_trial}")
print(f"Best test accuracy: {study.best_value}")
print(f"Best hyperparameters: {study.best_params}")

Best trial: FrozenTrial(number=3, state=TrialState.COMPLETE, values=[1.0], datetime_start=datetime.datetime(2023, 4, 18, 2, 41, 50, 324388), datetime_complete=datetime.datetime(2023, 4, 18, 2, 44, 19, 718464), params={'hidden_dim': 42, 'num_layers': 1, 'dropout': 0.010454300823487694, 'learning_rate': 0.05738663557134362, 'batch_size': 8, 'max_epochs': 18}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'hidden_dim': IntDistribution(high=128, log=False, low=16, step=1), 'num_layers': IntDistribution(high=3, log=False, low=1, step=1), 'dropout': FloatDistribution(high=0.5, log=False, low=0.0, step=None), 'learning_rate': FloatDistribution(high=0.1, log=True, low=1e-05, step=None), 'batch_size': CategoricalDistribution(choices=(8, 16, 32)), 'max_epochs': IntDistribution(high=30, log=False, low=10, step=1)}, trial_id=3, value=None)
Best test accuracy: 1.0
Best hyperparameters: {'hidden_dim': 42, 'num_layers': 1, 'dropout': 0.010454300823487694, 'learning_rate': 0.0

In [6]:
# FIXME: Check GPT

# # Train the best model with the best hyperparameters
# best_hyperparams = study.best_params
# hidden_dim = best_hyperparams['hidden_dim']
# num_layers = best_hyperparams['num_layers']
# dropout = best_hyperparams['dropout']
# learning_rate = best_hyperparams['learning_rate']
# # batch_size = best_hyperparams['batch_size']
# # max_epochs = best_hyperparams['max_epochs']

# # Define the model
# model = LSTMClassifier(
#     input_dim=2,
#     hidden_dim=hidden_dim,
#     output_dim=1,
#     num_layers=num_layers,
#     dropout=dropout,
#     learning_rate=learning_rate
# )


# # Load the data
# x = torch.randn(1000, input_dim)
# y = torch.randint(0, 2, (1000, 1)).float()
# dataset = MyDataset(x, y)
# train_size = int(len(dataset) * 0.8)
# val_size = len(dataset) - train_size
# train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
# train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
# val_loader = DataLoader(val_dataset, batch_size=batch_size)

# # Initialize the best model
# best_model = LSTMClassifier(input_dim, hidden_dim, output_dim)

# # Train the best model
# trainer = pl.Trainer(max_epochs=5, progress_bar_refresh_rate=0)
# trainer.fit(best_model, train_loader, val_loader)

# # Evaluate the best model on the test set
# test_size = len(val_dataset)
# test_loader = DataLoader(val_dataset, batch_size=batch_size)
# test_loss = trainer.test(best_model, test_dataloaders=test_loader)
# print(f"Test loss: {test_loss[0]['test_loss']:.3f}")


In [7]:
# # Load & show metrics
# metrics = pd.read_csv(f'./{DIR_NAME}/{EXP_NUM}/metrics.csv')
# train_loss = metrics[['train_loss', 'step', 'epoch']][~np.isnan(metrics['train_loss'])]
# val_loss = metrics[['val_loss', 'epoch']][~np.isnan(metrics['val_loss'])]
# test_loss = metrics['test_loss'].iloc[-1]

# fig, axes = plt.subplots(1, 2, figsize=(16, 5), dpi=200)
# axes[0].set_title('Train loss per batch')
# axes[0].plot(train_loss['step'], train_loss['train_loss'])
# axes[1].set_title('Val loss per epoch')
# axes[1].plot(val_loss['epoch'], val_loss['val_loss'])
# plt.savefig(f'./{DIR_NAME}/{EXP_NUM}/loss.png', dpi=200)
# plt.show(block = True)

# print('BCEWithLogits:')
# print(f"Train loss: {train_loss['train_loss'].iloc[-1]:.3f}")
# print(f"Val loss:   {val_loss['val_loss'].iloc[-1]:.3f}")
# print(f'Test loss:  {test_loss:.3f}')